In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import time # using sleep cannot avoid captcha
import pandas as pd

In [57]:
cases = pd.read_csv("data/ccapjdgmtoutputfile_20210706023712.csv")
#cases = pd.read_csv("data/ccapoutputfile_20210706023712.csv")
#cases['CaseNumber'][0]
cases

,CaseNumber,County,CountyName,Name,JudgmentDate,MultipleDebtors,DocketDate,Satisfaction,LastModifiedDate,ZipCodes
0,2021SC000268,45,Ozaukee,Latasha Eubanks,6/18/2021,True,6/18/2021,O,7/2/2021 15:08,53092
1,2020SC001080,36,Manitowoc,Kristie Martinez,6/15/2021,False,6/15/2021,No,6/15/2021 16:35,54220
2,2021SC000148,61,Trempealeau,Robert Luethi,6/16/2021,False,6/22/2021,No,6/22/2021 20:07,54747
3,2021SC000068,50,Price,David Ames,6/29/2021,False,6/29/2021,No,6/29/2021 18:49,54459
4,2021SC001257,30,Kenosha,R+D Investments LLC,6/28/2021,False,6/30/2021,No,7/6/2021 12:57,53142
...,...,...,...,...,...,...,...,...,...,...
80,2020SC003239,53,Rock,Barbara Congdon,6/2/2021,False,6/3/2021,No,6/3/2021 14:54,53563
81,2020SC001764,37,Marathon,Timothy Heymas,6/15/2021,True,6/15/2021,No,6/15/2021 21:02,54455
82,2017SC000379,70,Winnebago,Speed Investments LLC,6/29/2021,False,6/29/2021,No,6/29/2021 14:30,54901
83,2021SC002797,40,Milwaukee,Donald & Virginia Kretlow Revocable Trust,6/8/2021,False,7/2/2021,No,7/2/2021 21:47,53209


In [2]:
# r = requests.get("https://wcca.wicourts.gov/case.html")
# r.raise_for_status()
# html = r.text
# doc = BeautifulSoup(html, "html.parser")

In [17]:
options = Options()
options.headless = False
driver = webdriver.Chrome(options=options, executable_path='./chromedriver')
driver.implicitly_wait(1)
driver.get("https://wcca.wicourts.gov/case.html")
type(driver.page_source) # return the full page HTML code

str

### Code below will encounter CAPTCHA for first trial

In [59]:
file_num = 1
case_number = cases['CaseNumber'][0]
county = cases['County'][0]
textbox = driver.find_element_by_name('caseNo').send_keys(case_number)
span = driver.find_element_by_xpath("//span[@class='Select-arrow']").click()
county_choose = driver.find_element_by_xpath("//input[@class='Select-input']").send_keys(cases['CountyName'][0])

finish_choose = driver.find_element_by_xpath("//*[@class='Select-menu-outer']").click()
submit = driver.find_element_by_xpath("//button[@type='submit']").click()
time.sleep(2)
#scrape here

In [60]:
#download the html source file and go back
html_source = driver.page_source
filename = 'case_html/' + str(file_num) + 'case.csv'
with open(filename,'w') as file:
    file.write(html_source)
    file.write('\n')

driver.back()

In [61]:
for i in range(1, 85):
    case_number = cases['CaseNumber'][i]
    county = cases['County'][i]
    textbox = driver.find_element_by_name('caseNo')
    textbox.clear()
    textbox.send_keys(case_number)
    span = driver.find_element_by_xpath("//span[@class='Select-arrow']").click()
    county_choose = driver.find_element_by_xpath("//input[@class='Select-input']").send_keys(cases['CountyName'][i])

    finish_choose = driver.find_element_by_xpath("//*[@class='Select-menu-outer']").click()
    submit = driver.find_element_by_xpath("//button[@type='submit']").click()
    time.sleep(2)
    
    # scrape the data and store the data here
    html_source = driver.page_source
    filename = 'case_html/' + str(i+1) + 'case.csv'
    with open(filename,'w') as file:
        file.write(html_source)
        file.write('\n')
    time.sleep(1)
    driver.back()
    time.sleep(2)

ElementClickInterceptedException: Message: element click intercepted: Element <span class="Select-arrow"></span> is not clickable at point (264, 487). Other element would receive the click: <div style="width: 100%; height: 100%; position: fixed; pointer-events: auto; top: 0px; left: 0px; z-index: 0; background-color: rgb(255, 255, 255); opacity: 0.05; cursor: pointer;"></div>
  (Session info: chrome=95.0.4638.69)


In [20]:
# from IPython.core.display import Image
# driver.save_screenshot("shot1.png")
# Image("shot1.png")